In [2]:
# CELL 1: Load CSV and Basic Setup
import pandas as pd
from pathlib import Path
import nibabel as nib
import numpy as np

# Load subject info from CSV
CSV_FILE = Path('/user_data/csimmon2/git_repos/long_pt/long_pt_sub_info.csv')
df = pd.read_csv(CSV_FILE)

# Session start mapping (for special cases)
SESSION_START = {'sub-010': 2, 'sub-018': 2, 'sub-068': 2}

print("✓ CSV loaded and configuration set")

✓ CSV loaded and configuration set


In [12]:
# CELL 2: Paths, Configuration, and Subject Loading Functions
def load_subjects_by_group(group_filter=None, patient_only=True):
    """Fixed version with proper hemisphere mapping"""
    
    filtered_df = df.copy()
    
    if patient_only is True:
        filtered_df = filtered_df[filtered_df['patient'] == 1]
    elif patient_only is False:
        filtered_df = filtered_df[filtered_df['patient'] == 0]
    
    if group_filter:
        if isinstance(group_filter, str):
            group_filter = [group_filter]
        filtered_df = filtered_df[filtered_df['group'].isin(group_filter)]
    
    subjects = {}
    
    for _, row in filtered_df.iterrows():
        subject_id = row['sub']
        
        subj_dir = BASE_DIR / subject_id
        if not subj_dir.exists():
            continue
            
        sessions = []
        for ses_dir in subj_dir.glob('ses-*'):
            if ses_dir.is_dir():
                sessions.append(ses_dir.name.replace('ses-', ''))
        
        if not sessions:
            continue
            
        sessions = sorted(sessions, key=lambda x: int(x))
        start_session = SESSION_START.get(subject_id, 1)
        available_sessions = [s for s in sessions if int(s) >= start_session]
        
        if not available_sessions:
            continue
            
        # FIX: Convert 'left'/'right' to 'l'/'r' to match file naming
        hemisphere_full = row.get('intact_hemi', 'left') if pd.notna(row.get('intact_hemi', None)) else 'left'
        hemisphere = 'l' if hemisphere_full == 'left' else 'r'  # Convert to single letter
        
        subjects[subject_id] = {
            'code': f"{row['group']}{subject_id.split('-')[1]}",
            'sessions': available_sessions,
            'hemi': hemisphere,  # Now uses 'l' or 'r' 
            'group': row['group'],
            'patient_status': 'patient' if row['patient'] == 1 else 'control',
            'age_1': row['age_1'] if pd.notna(row['age_1']) else None
        }
    
    return subjects

# Reload with correct hemisphere mapping
ALL_PATIENTS = load_subjects_by_group(group_filter=None, patient_only=True)
OTC_PATIENTS = load_subjects_by_group(group_filter='OTC', patient_only=True)
NON_OTC_PATIENTS = load_subjects_by_group(group_filter='nonOTC', patient_only=True)
ALL_CONTROLS = load_subjects_by_group(group_filter=None, patient_only=False)

# Update analysis subjects
ANALYSIS_SUBJECTS = ALL_PATIENTS

print("FIXED HEMISPHERE MAPPING:")
for subj_id, info in list(ANALYSIS_SUBJECTS.items())[:3]:
    print(f"  {subj_id}: hemisphere = '{info['hemi']}'")

FIXED HEMISPHERE MAPPING:
  sub-004: hemisphere = 'l'
  sub-007: hemisphere = 'r'
  sub-008: hemisphere = 'l'


In [13]:
# CELL 3: Category-Specific Mask Creation (Updated for Groups)
from scipy.ndimage import binary_dilation

def create_category_specific_masks(subject_id, subjects_dict, dilation_iterations=1):
    """
    Create separate anatomical search masks for each category
    Updated to use dynamic subjects dictionary
    """
    if subject_id not in subjects_dict:
        print(f"❌ {subject_id} not in analysis group")
        return {}
        
    info = subjects_dict[subject_id]
    code = info['code']
    hemi = info['hemi']
    first_session = info['sessions'][0]
    
    roi_dir = BASE_DIR / subject_id / f'ses-{first_session}' / 'ROIs'
    
    print(f"\n{'='*70}")
    print(f"{code} - Category-Specific Masks (1x dilation) [ses-{first_session}] [{info['group']} {info['patient_status']}]")
    print(f"{'='*70}")
    
    category_masks = {}
    
    for category, parcel_list in CATEGORY_PARCELS.items():
        print(f"\n{category.upper()}:")
        
        combined = None
        ref_img = None
        
        for parcel in parcel_list:
            parcel_file = roi_dir / f'{hemi}_{parcel}_mask.nii.gz'
            
            if not parcel_file.exists():
                print(f"  ⚠️  Missing: {parcel}")
                continue
            
            img = nib.load(parcel_file)
            mask = img.get_fdata() > 0
            n_vox = np.sum(mask)
            print(f"  {parcel:20s}: {n_vox:6d} voxels")
            
            if combined is None:
                combined = mask
                ref_img = img
            else:
                combined = combined | mask
        
        if combined is None:
            print(f"  ❌ No parcels found")
            continue
        
        print(f"  Combined (before dilation): {np.sum(combined):6d} voxels")
        
        # Single dilation
        combined = binary_dilation(combined, iterations=dilation_iterations)
        print(f"  After 1x dilation:          {np.sum(combined):6d} voxels")
        
        # Save to first session directory
        output_file = roi_dir / f'{hemi}_{category}_searchmask.nii.gz'
        mask_img = nib.Nifti1Image(combined.astype(np.float32), ref_img.affine)
        nib.save(mask_img, output_file)
        
        category_masks[category] = output_file
        print(f"  ✓ Saved: {output_file.name}")
    
    return category_masks

# Create category masks for selected analysis group
print("\n" + "="*70)
print(f"CREATING CATEGORY MASKS FOR ANALYSIS GROUP")
print("="*70)

category_masks = {}
for subject_id in ANALYSIS_SUBJECTS.keys():
    category_masks[subject_id] = create_category_specific_masks(subject_id, ANALYSIS_SUBJECTS, dilation_iterations=1)

print(f"\n✓ Category masks created for {len(category_masks)} subjects")


CREATING CATEGORY MASKS FOR ANALYSIS GROUP

OTC004 - Category-Specific Masks (1x dilation) [ses-01] [OTC patient]

FACE:
  fusiform            :  12134 voxels
  Combined (before dilation):  12134 voxels
  After 1x dilation:           19331 voxels
  ✓ Saved: l_face_searchmask.nii.gz

WORD:
  fusiform            :  12134 voxels
  Combined (before dilation):  12134 voxels
  After 1x dilation:           19331 voxels
  ✓ Saved: l_word_searchmask.nii.gz

OBJECT:
  lateraloccipital    :  15770 voxels
  Combined (before dilation):  15770 voxels
  After 1x dilation:           25843 voxels
  ✓ Saved: l_object_searchmask.nii.gz

HOUSE:
  parahippocampal     :   1847 voxels
  lingual             :   9576 voxels
  isthmuscingulate    :   3317 voxels
  Combined (before dilation):  14740 voxels
  After 1x dilation:           24507 voxels
  ✓ Saved: l_house_searchmask.nii.gz

nonOTC007 - Category-Specific Masks (1x dilation) [ses-01] [nonOTC patient]

FACE:
  fusiform            :  13276 voxels
  Com

In [ ]:
# CELL 4: Resample Individual Parcels to FSL Space
from nilearn.image import resample_to_img

def resample_parcels_to_fsl_space(subject_id):
    """Resample individual parcels from FreeSurfer space to FSL space"""
    
    info = ANALYSIS_SUBJECTS[subject_id]
    code = info['code']
    hemi = info['hemi']
    first_session = info['sessions'][0]
    
    roi_dir = BASE_DIR / subject_id / f'ses-{first_session}' / 'ROIs'
    anat_file = BASE_DIR / subject_id / f'ses-{first_session}' / 'anat' / f'{subject_id}_ses-{first_session}_T1w_brain.nii.gz'
    
    print(f"\n{code} - Resampling parcels to FSL space:")
    print("="*50)
    
    if not anat_file.exists():
        print(f"❌ FSL anatomy not found: {anat_file}")
        return
    
    anat_img = nib.load(anat_file)
    print(f"Target FSL anatomy shape: {anat_img.shape}")
    
    # All parcels that might exist
    parcels = ['fusiform', 'lateraloccipital', 'parahippocampal', 'inferiortemporal', 
               'middletemporal', 'lingual', 'isthmuscingulate']
    
    resampled_count = 0
    
    for parcel in parcels:
        parcel_file = roi_dir / f'{hemi}_{parcel}_mask.nii.gz'
        
        if parcel_file.exists():
            parcel_img = nib.load(parcel_file)
            original_voxels = np.sum(parcel_img.get_fdata() > 0)
            
            if parcel_img.shape != anat_img.shape:
                print(f"  {parcel:20s}: {parcel_img.shape} → {anat_img.shape}")
                
                # Resample to FSL space
                resampled = resample_to_img(parcel_img, anat_img, interpolation='nearest')
                resampled_voxels = np.sum(resampled.get_fdata() > 0)
                
                # Overwrite with resampled version
                nib.save(resampled, parcel_file)
                print(f"    Voxels: {original_voxels:,} → {resampled_voxels:,}")
                resampled_count += 1
            else:
                print(f"  {parcel:20s}: Already correct shape ✓")
        else:
            print(f"  {parcel:20s}: Not found")
    
    print(f"\n✓ Resampled {resampled_count} parcels for {code}")

# Resample parcels for all subjects
print("\n" + "="*70)
print("RESAMPLING INDIVIDUAL PARCELS TO FSL SPACE")
print("="*70)

for subject_id in ANALYSIS_SUBJECTS.keys():
    resample_parcels_to_fsl_space(subject_id)

print("\n✓ All parcels resampled to FSL space!")


RESAMPLING INDIVIDUAL PARCELS TO FSL SPACE

OTC004 - Resampling parcels to FSL space:
Target FSL anatomy shape: (176, 256, 256)
  fusiform            : (256, 256, 256) → (176, 256, 256)
    Voxels: 12,134 → 12,134
  lateraloccipital    : (256, 256, 256) → (176, 256, 256)
    Voxels: 15,770 → 15,770
  parahippocampal     : (256, 256, 256) → (176, 256, 256)
    Voxels: 1,847 → 1,847
  inferiortemporal    : (256, 256, 256) → (176, 256, 256)
    Voxels: 14,609 → 14,609
  middletemporal      : (256, 256, 256) → (176, 256, 256)
    Voxels: 15,030 → 15,030
  lingual             : (256, 256, 256) → (176, 256, 256)
    Voxels: 9,576 → 9,576
  isthmuscingulate    : (256, 256, 256) → (176, 256, 256)
    Voxels: 3,317 → 3,317

✓ Resampled 7 parcels for OTC004

nonOTC007 - Resampling parcels to FSL space:
Target FSL anatomy shape: (176, 256, 256)
  fusiform            : (256, 256, 256) → (176, 256, 256)
    Voxels: 13,276 → 13,276
  lateraloccipital    : (256, 256, 256) → (176, 256, 256)
    Voxel

In [27]:
# CELL 5: Recreate Category Masks from Resampled Parcels
# Remove old category masks and recreate from properly resampled parcels

def recreate_category_masks_from_resampled(subject_id):
    """Recreate category masks from the newly resampled parcels"""
    
    info = ANALYSIS_SUBJECTS[subject_id]
    code = info['code']
    hemi = info['hemi']
    first_session = info['sessions'][0]
    roi_dir = BASE_DIR / subject_id / f'ses-{first_session}' / 'ROIs'
    
    print(f"\n{code} - Recreating category masks from resampled parcels:")
    
    # Remove old search masks
    for old_mask in roi_dir.glob(f"{hemi}_*_searchmask*.nii.gz"):
        old_mask.unlink()
        print(f"  Removed: {old_mask.name}")
    
    # Recreate using cell 3 logic
    category_masks = create_category_specific_masks(subject_id, ANALYSIS_SUBJECTS, dilation_iterations=1)
    
    return category_masks

# Recreate category masks for all subjects
print("\n" + "="*70)  
print("RECREATING CATEGORY MASKS FROM RESAMPLED PARCELS")
print("="*70)

category_masks = {}
for subject_id in ANALYSIS_SUBJECTS.keys():
    category_masks[subject_id] = recreate_category_masks_from_resampled(subject_id)

print("\n✓ Category masks recreated from properly resampled parcels!")


RECREATING CATEGORY MASKS FROM RESAMPLED PARCELS

OTC004 - Recreating category masks from resampled parcels:
  Removed: l_face_searchmask.nii.gz
  Removed: l_object_searchmask.nii.gz
  Removed: l_word_searchmask.nii.gz
  Removed: l_face_searchmask_func.nii.gz
  Removed: l_house_searchmask_func.nii.gz
  Removed: l_word_searchmask_func.nii.gz
  Removed: l_house_searchmask.nii.gz
  Removed: l_object_searchmask_func.nii.gz

OTC004 - Category-Specific Masks (1x dilation) [ses-01] [OTC patient]

FACE:
  fusiform            :  12134 voxels
  Combined (before dilation):  12134 voxels
  After 1x dilation:           19331 voxels
  ✓ Saved: l_face_searchmask.nii.gz

WORD:
  fusiform            :  12134 voxels
  Combined (before dilation):  12134 voxels
  After 1x dilation:           19331 voxels
  ✓ Saved: l_word_searchmask.nii.gz

OBJECT:
  lateraloccipital    :  15770 voxels
  Combined (before dilation):  15770 voxels
  After 1x dilation:           25843 voxels
  ✓ Saved: l_object_searchmask.n

In [ ]:
# CELL 6: Functional ROI Extraction (Updated and Cleaned)
from scipy.ndimage import label, center_of_mass

def extract_functional_rois_final(subject_id, threshold_z=2.3):
    """
    Extract functional cluster ROIs across all sessions
    Uses category-specific anatomical masks (now properly resampled to FSL space)
    """
    
    # Get subject info from ANALYSIS_SUBJECTS
    info = ANALYSIS_SUBJECTS[subject_id]
    code = info['code']
    hemi = info['hemi']
    sessions = info['sessions']
    first_session = sessions[0]
    
    print(f"\n{'='*70}")
    print(f"{code} - Functional Clusters [masks from ses-{first_session}, z>{threshold_z}] [{info['group']} {info['patient_status']}]")
    print(f"{'='*70}")
    
    all_results = {}
    
    # Process each category using COPE_MAP
    for category, cope_num in COPE_MAP.items():
        print(f"\n{category.upper()}:")
        all_results[category] = {}
        
        # Load category-specific mask (now in correct FSL space)
        mask_file = BASE_DIR / subject_id / f'ses-{first_session}' / 'ROIs' / f'{hemi}_{category}_searchmask.nii.gz'
        if not mask_file.exists():
            print(f"  ⚠️  Mask not found: {mask_file}")
            continue
        
        mask = nib.load(mask_file).get_fdata() > 0
        affine = nib.load(mask_file).affine
        
        # Process each session
        for session in sessions:
            feat_dir = BASE_DIR / subject_id / f'ses-{session}' / 'derivatives' / 'fsl' / 'loc' / 'HighLevel.gfeat'
            
            # Select correct zstat file (first session vs registered to first session)
            zstat_file = 'zstat1.nii.gz' if session == first_session else f'zstat1_ses{first_session}.nii.gz'
            cope_file = feat_dir / f'cope{cope_num}.feat' / 'stats' / zstat_file
            
            if not cope_file.exists():
                print(f"  ses-{session}: cope file missing ({zstat_file})")
                continue
            
            # Load functional activation
            zstat = nib.load(cope_file).get_fdata()
            
            # Apply threshold and mask (should now have matching dimensions)
            suprathresh = (zstat > threshold_z) & mask
            
            if suprathresh.sum() < 50:
                print(f"  ses-{session}: <50 voxels (skipped)")
                continue
            
            # Find connected components
            labeled, n_clusters = label(suprathresh)
            if n_clusters == 0:
                print(f"  ses-{session}: No clusters found")
                continue
            
            # Get largest cluster
            cluster_sizes = [(labeled == i).sum() for i in range(1, n_clusters + 1)]
            largest_idx = np.argmax(cluster_sizes) + 1
            roi_mask = (labeled == largest_idx)
            
            # Extract metrics
            peak_idx = np.unravel_index(np.argmax(zstat * roi_mask), zstat.shape)
            peak_z = zstat[peak_idx]
            centroid = nib.affines.apply_affine(affine, center_of_mass(roi_mask))
            
            # Store results
            all_results[category][session] = {
                'n_voxels': cluster_sizes[largest_idx - 1],
                'peak_z': peak_z,
                'centroid': centroid,
                'roi_mask': roi_mask
            }
            
            print(f"  ses-{session}: {cluster_sizes[largest_idx - 1]:4d} voxels, peak z={peak_z:.2f}")
    
    return all_results

# Extract functional ROIs for all subjects in analysis group
print("\n" + "="*70)
print("EXTRACTING FUNCTIONAL ROIs - CORRECTED MASKS")
print("="*70)

golarai_functional_final = {}

for subject_id in ANALYSIS_SUBJECTS.keys():
    golarai_functional_final[subject_id] = extract_functional_rois_final(subject_id, threshold_z=2.3)

print(f"\n✓ Functional ROI extraction complete for {len(golarai_functional_final)} subjects!")


EXTRACTING FUNCTIONAL ROIs - CORRECTED MASKS

OTC004 - Functional Clusters [masks from ses-01, z>2.3] [OTC patient]

FACE:
  ses-01:  352 voxels, peak z=3.50
  ses-02: 2834 voxels, peak z=6.81
  ses-03: 4282 voxels, peak z=8.76
  ses-05: 3798 voxels, peak z=8.69
  ses-06: 4068 voxels, peak z=13.33
  ses-07: cope file missing (zstat1_ses01.nii.gz)

WORD:
  ses-01: <50 voxels (skipped)
  ses-02: 1037 voxels, peak z=5.59
  ses-03: 1891 voxels, peak z=6.24
  ses-05:  858 voxels, peak z=6.32
  ses-06:  510 voxels, peak z=6.97
  ses-07: cope file missing (zstat1_ses01.nii.gz)

OBJECT:
  ses-01: 3747 voxels, peak z=6.77
  ses-02: 6577 voxels, peak z=7.90
  ses-03: 8011 voxels, peak z=8.62
  ses-05: 10690 voxels, peak z=10.12
  ses-06: 7137 voxels, peak z=11.29
  ses-07: cope file missing (zstat1_ses01.nii.gz)

HOUSE:
  ses-01: 1379 voxels, peak z=5.25
  ses-02: 1998 voxels, peak z=9.42
  ses-03: 3109 voxels, peak z=11.75
  ses-05: 4933 voxels, peak z=12.23
  ses-06: 4691 voxels, peak z=12.64

In [ ]:
# CELL 6: Analysis Summary and Group Comparison
def summarize_analysis_results(functional_results, subjects_dict):
    """
    Create summary of analysis results by group
    """
    
    print("\n" + "="*70)
    print("ANALYSIS SUMMARY BY GROUP")
    print("="*70)
    
    # Group by patient status and group
    groups = {}
    for subject_id, info in subjects_dict.items():
        group_key = f"{info['group']}_{info['patient_status']}"
        if group_key not in groups:
            groups[group_key] = []
        groups[group_key].append(subject_id)
    
    # Summary by category
    for category in CATEGORY_PARCELS.keys():
        print(f"\n{category.upper()} CATEGORY:")
        print("-" * 50)
        
        for group_key, subject_list in groups.items():
            print(f"\n{group_key}:")
            
            session_data = {}
            for subject_id in subject_list:
                if subject_id in functional_results and category in functional_results[subject_id]:
                    for session, data in functional_results[subject_id][category].items():
                        if session not in session_data:
                            session_data[session] = []
                        session_data[session].append(data['n_voxels'])
            
            for session in sorted(session_data.keys()):
                voxels = session_data[session]
                if voxels:
                    mean_voxels = np.mean(voxels)
                    print(f"  ses-{session}: {len(voxels)} subjects, mean={mean_voxels:.1f} voxels")
    
    # Subject completion rates
    print(f"\n{'='*70}")
    print("SUBJECT COMPLETION RATES")
    print(f"{'='*70}")
    
    for subject_id, info in subjects_dict.items():
        code = info['code']
        group_status = f"{info['group']} {info['patient_status']}"
        
        if subject_id in functional_results:
            categories_found = len(functional_results[subject_id])
            sessions_per_category = {}
            
            for cat, sessions in functional_results[subject_id].items():
                sessions_per_category[cat] = len(sessions)
            
            print(f"{code:10s} ({group_status:15s}): {categories_found}/4 categories, sessions: {dict(sessions_per_category)}")
        else:
            print(f"{code:10s} ({group_status:15s}): No functional data")

# Generate summary for current analysis
summarize_analysis_results(golarai_functional_final, ANALYSIS_SUBJECTS)

print(f"\n✓ Analysis complete for {len(ANALYSIS_SUBJECTS)} subjects")
print(f"✓ Data ready for RSA analysis")
print(f"✓ Results saved to: {OUTPUT_DIR}")

In [ ]:
# CELL 7: Calculate peak drift (Updated for Dynamic Subjects)
def calculate_peak_drift(functional_results, subjects_dict):
    """Calculate total drift for each category using dynamic subjects"""
    print("\nPeak Movement Summary:")
    print(f"{'Subject':<10} {'Category':<8} {'Total Drift (mm)':<18} {'Sessions'}")
    print("-"*75)
    
    for subj in subjects_dict.keys():  # Use dynamic subjects
        info = subjects_dict[subj]
        code = info['code']
        
        if subj not in functional_results:
            continue
            
        for cat in ['face', 'word', 'object', 'house']:
            if cat not in functional_results[subj]:
                continue
            sessions = sorted(functional_results[subj][cat].keys())
            if len(sessions) < 2:
                continue
            
            first = functional_results[subj][cat][sessions[0]]['centroid']
            last = functional_results[subj][cat][sessions[-1]]['centroid']
            drift = np.linalg.norm(last - first)
            
            group_status = f"{info['group']}/{info['patient_status']}"
            print(f"{code:<10} {cat:<8} {drift:>10.1f} mm        {', '.join(sessions)} [{group_status}]")

calculate_peak_drift(golarai_functional_final, ANALYSIS_SUBJECTS)


Peak Movement Summary:
Subject    Category Total Drift (mm)   Sessions
---------------------------------------------------------------------------


In [ ]:
# CELL 8: Multi-category peak visualization (All Categories on One Anatomy)
from nilearn import plotting, image
import matplotlib.pyplot as plt
import numpy as np

def plot_subject_all_peaks_ortho(subject_id, functional_results, subjects_dict):
    """Plot ALL category peaks on one orthogonal anatomy view"""
    
    if subject_id not in subjects_dict:
        print(f"Subject {subject_id} not in analysis group")
        return
        
    info = subjects_dict[subject_id]
    code = info['code']
    first_session = info['sessions'][0]
    
    # Load anatomy
    anat_path = BASE_DIR / subject_id / f'ses-{first_session}' / 'anat' / f'{subject_id}_ses-{first_session}_T1w_brain.nii.gz'
    
    if not anat_path.exists():
        print(f"No anatomy for {code}: {anat_path}")
        return
    
    anat_img = image.load_img(anat_path)
    
    if subject_id not in functional_results:
        print(f"No functional results for {code}")
        return
    
    # Color mapping
    color_map = {'face': 'red', 'word': 'blue', 'object': 'green', 'house': 'orange'}
    
    # Collect ALL peaks from ALL categories
    all_peaks = []
    peak_colors = []
    peak_labels = []
    
    for category in ['face', 'word', 'object', 'house']:
        if category not in functional_results[subject_id]:
            continue
        
        for session in functional_results[subject_id][category]:
            peak = functional_results[subject_id][category][session]['centroid']
            all_peaks.append(peak)
            peak_colors.append(color_map[category])
            peak_labels.append(f"{category}-s{session}")
    
    if len(all_peaks) == 0:
        print(f"No peaks found for {code}")
        return
    
    # Use center of all peaks for view
    all_peaks_array = np.array(all_peaks)
    center_coord = np.mean(all_peaks_array, axis=0)
    
    # Create figure
    fig = plt.figure(figsize=(12, 10))
    
    # Create orthographic view
    display = plotting.plot_anat(
        anat_img,
        display_mode='ortho',
        cut_coords=(center_coord[0], center_coord[1], center_coord[2]),
        draw_cross=False,
        annotate=False,
        black_bg=True,
        title=f'{code} - All Category Peaks [{info["group"]} {info["patient_status"]}]'
    )
    
    # Add peaks by category (so we can control colors)
    for category in ['face', 'word', 'object', 'house']:
        if category not in functional_results[subject_id]:
            continue
        
        category_peaks = []
        category_sessions = []
        
        for session in functional_results[subject_id][category]:
            peak = functional_results[subject_id][category][session]['centroid']
            category_peaks.append(peak)
            category_sessions.append(session)
        
        if len(category_peaks) > 0:
            display.add_markers(
                np.array(category_peaks),
                marker_color=color_map[category],
                marker_size=12
            )
    
    # Create legend and summary info
    legend_text = []
    for category in ['face', 'word', 'object', 'house']:
        if category in functional_results[subject_id]:
            n_sessions = len(functional_results[subject_id][category])
            legend_text.append(f"{category.upper()}: {n_sessions} sessions")
    
    info_text = " | ".join(legend_text)
    plt.figtext(0.5, 0.02, info_text, ha='center', fontsize=10, color='white')
    
    # Color legend
    legend_text2 = "RED=Face | BLUE=Word | GREEN=Object | ORANGE=House"
    plt.figtext(0.5, 0.95, legend_text2, ha='center', fontsize=10, color='white')
    
    plt.subplots_adjust(top=0.88, bottom=0.08)
    
    # Save
    output_file = OUTPUT_DIR / f'{code}_all_peaks_ortho.png'
    plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='black')
    plt.show()
    
    print(f"✓ Saved: {code}_all_peaks_ortho.png")

# Generate multi-category peak plots for all subjects  
print("\n" + "="*70)
print("GENERATING MULTI-CATEGORY PEAK VISUALIZATIONS")
print("="*70)

for subject_id in ANALYSIS_SUBJECTS.keys():
    info = ANALYSIS_SUBJECTS[subject_id]
    code = info['code']
    print(f"\n{code} - All categories on one view:")
    plot_subject_all_peaks_ortho(subject_id, golarai_functional_final, ANALYSIS_SUBJECTS)

print("\n✓ All multi-category visualizations complete!")


GENERATING MULTI-CATEGORY PEAK VISUALIZATIONS

OTC004 - All categories on one view:
No peaks found for OTC004

nonOTC007 - All categories on one view:
No peaks found for nonOTC007

OTC008 - All categories on one view:
No peaks found for OTC008

OTC010 - All categories on one view:
No peaks found for OTC010

OTC017 - All categories on one view:
No peaks found for OTC017

OTC021 - All categories on one view:
No peaks found for OTC021

nonOTC045 - All categories on one view:
No peaks found for nonOTC045

nonOTC047 - All categories on one view:
No peaks found for nonOTC047

nonOTC049 - All categories on one view:
No peaks found for nonOTC049

nonOTC070 - All categories on one view:
No peaks found for nonOTC070

nonOTC072 - All categories on one view:
No peaks found for nonOTC072

nonOTC073 - All categories on one view:
No peaks found for nonOTC073

OTC079 - All categories on one view:
No peaks found for OTC079

nonOTC081 - All categories on one view:
No peaks found for nonOTC081

nonOTC08